# Policy Iteration

- Policy Evaluation
- Policy Improvement

In [ ]:
from gridworld import GridworldEnv
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Policy Evaluation

In [ ]:
# Policy evaluation
def policy_evaluation(policy, env, discount_factor=1.0, theta=0.00001):
    """
    Evaluate a policy given an environment.

    Args:
        policy:[S, A]shaped matrix representing the policy. Random in our case
        env: OpenAI env. env.P -> transition dynamics of the environment.
            env.P[s][a] [(prob, next_state, reward, done)].
            env.nS is number of states in the environment.
            env.nA is number of actions in the environment.
        theta: Stop iteration once value function change is
            less than theta for all states.
        discount_factor: Gamma discount factor.

    Returns:
        Vector of length env.nS representing the value function.
    """
    print("Running policy evaluation...")
    # Start with a (all 0) value function
    V = np.zeros(env.nS)
    V_new = np.copy(V)
    while True:
        delta = 0
        # For each state, perform a "backup"
        for s in range(env.nS):
            v = 0
            # Look at the possible next actions
            for a, pi_a in enumerate(policy[s]):
                # For each action, look at the possible next states...
                for prob, next_state, reward, done in env.P[s][a]:
                    # Calculate the expected value as per backup diagram
                    v += pi_a * prob * (reward + discount_factor * V[next_state])
            # How much our value function changed (across any states)
            V_new[s] = v
            delta = max(delta, np.abs(V_new[s] - V[s]))
        V = np.copy(V_new)
        # Stop if change is below a threshold
        if delta < theta:
            break
    return np.array(V)

In [ ]:
# Custom print to show state values inside the grid
def grid_print(V, k=None):
    ax = sns.heatmap(
        V.reshape(env.shape),
        annot=True,
        square=True,
        cbar=False,
        cmap="Blues",
        xticklabels=False,
        yticklabels=False,
    )

    if k:
        ax.set(title="K = {0}".format(k))
    plt.show()

In [ ]:
env = GridworldEnv()
env.reset()

In [ ]:
# Create a random policy
random_policy = np.ones([env.nS, env.nA]) / env.nA
print(random_policy)

In [ ]:
# run policy iteration for random policy
V_pi = policy_evaluation(random_policy, env, discount_factor=1.0, theta=0.00001)

# Print policy
grid_print(V_pi.reshape(env.shape))

# Policy Improvement

In [ ]:
# Policy Improvement
def policy_improvement(policy, V, env, discount_factor=1.0):
    """
    Improve a policy given an environment and a full description
    of the environment's dynamics and the state-values V.

    Args:
        policy: [S, A] shaped matrix representing the policy.
        V: current state-value for the given policy
        env: OpenAI env. env.P -> transition dynamics of the environment.
            env.P[s][a] [(prob, next_state, reward, done)].
            env.nS is number of states in the environment.
            env.nA is number of actions in the environment.
        discount_factor: Gamma discount factor.

    Returns:
        policy: [S, A] shaped matrix representing improved policy.
        policy_changed: boolean which has value of `True` if there
                        was a change in policy
    """

    def argmax_a(arr):
        """
        Return idxs of all max values in an array.
        """
        max_idx = []
        max_val = float("-inf")
        for idx, elem in enumerate(arr):
            if elem == max_val:
                max_idx.append(idx)
            elif elem > max_val:
                max_idx = [idx]
                max_val = elem
        return max_idx

    print("Running policy improvement...")
    policy_changed = False
    Q = np.zeros([env.nS, env.nA])
    new_policy = np.zeros([env.nS, env.nA])

    # For each state, perform a "greedy improvement"
    for s in range(env.nS):
        old_action = np.array(policy[s])
        for a in range(env.nA):
            for prob, next_state, reward, done in env.P[s][a]:
                # Calculate the expected value as per backup diagram
                Q[s, a] += prob * (reward + discount_factor * V[next_state])

        # get maximizing actions and set new policy for state s
        best_actions = argmax_a(Q[s])
        new_policy[s, best_actions] = 1.0 / len(best_actions)

    if not np.allclose(new_policy[s], policy[s]):
        policy_changed = True

    return new_policy, policy_changed

In [ ]:
new_policy, changed = policy_improvement(random_policy, V_pi, env, discount_factor=1.0)

In [ ]:
new_policy

In [ ]:
# Evaluate the new policy
V_pi = policy_evaluation(new_policy, env, discount_factor=1.0, theta=0.00001)

# Print value
grid_print(V_pi.reshape(env.shape))

# Policy Iteration

In [ ]:
def policy_iteration(env, discount_factor=1.0, theta=0.00001):

    # initialize a random policy
    policy = np.ones([env.nS, env.nA]) / env.nA
    while True:
        V = policy_evaluation(policy, env, discount_factor, theta)
        policy, changed = policy_improvement(policy, V, env, discount_factor)
        if not changed:  # terminate iteration once no improvement is observed
            V_optimal = policy_evaluation(policy, env, discount_factor, theta)
            print("Optimal Policy\n", policy)
            return np.array(V_optimal), policy

In [ ]:
# Run policy iteration on Grid world
V_star, pi_star = policy_iteration(env)

# Print optimal policy state values
grid_print(V_star.reshape(env.shape))

In [ ]:
import pickle
with open('S03_pi_star.pkl', 'wb') as file:
    pickle.dump(dict(policy=pi_star), file)